# Import Demand

In [424]:
import math

sentido = 'ida' # ida ou volta
vinculo = 'funcionario' # aluno ou funcionario
terminal = 'p3' # butanta ou p3
horario = '8h' # 8h e 18h

map_file_name = 'data/disciplinas/' + sentido + '_' + vinculo + '_' + terminal + '_' + horario + '.json'

acao = 'embarque' if sentido == 'volta' else 'desembarque' # dump final
contrario_acao = 'desembarque' if acao == 'embarque' else 'embarque' # filtro

hora = 1080 if horario == '18h' else 480
def intervalo_horario_am_pm(horario_pesquisa):
    if isinstance(horario_pesquisa, float) and math.isnan(horario_pesquisa):
        return False
    if horario == '18h':
        return '5:30:00 PM' <= horario_pesquisa <= '7:30:00 PM'
    else:
        return '6:30:00 AM' <= horario_pesquisa <= '8:30:00 AM'
def intervalo_horario_inicio_matrusp(horario_inicio, horario_fim):
    if horario == '18h':
        return '15:30' <= horario_inicio <= '17:30'
    else:
        return '07:30' <= horario_inicio <= '08:30'
def intervalo_horario_fim_matrusp(horario_inicio, horario_fim):
    return '17:30' <= horario_fim <= '19:00'
intervalo_horario = intervalo_horario_inicio_matrusp if sentido == 'ida' else intervalo_horario_fim_matrusp

def is_butanta(x):
    return x == 'Ponto de ônibus metrô Butantã' or x == 'Ponto de ônibus - Acesso CPTM I'
def is_p3(x):
    return x == 'Ponto de ônibus Portaria III' or x == 'Ponto de ônibus acesso Vl. Indiana'
filtra_terminal = is_butanta if terminal == 'butanta' else is_p3

def get_dados_vinculo(aulas_alunos, funcionarios_intervalo_horario):
    if vinculo == 'aluno':
        return aulas_alunos
    return funcionarios_intervalo_horario
def grad_ou_pos(vinculo):
    return vinculo == 'Aluno de graduação' or vinculo == 'Aluno de pós-graduação'
def funcionario(vinculo):
    return vinculo == 'Funcionário'
filtra_vinculo = grad_ou_pos if vinculo == 'aluno' else funcionario

dias = ['seg', 'ter', 'qua', 'qui', 'sex', 'sab', 'dom']

# Aulas MATRUSP

##### Read jsons

In [425]:
import os
import json

In [426]:
files = []
dir_path = 'data/disciplinas/json/'
for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        files.append(path)

In [427]:
len(files)

5205

In [428]:
disciplinas = []
for file in files:
    with open(dir_path + file, 'r+') as f:
        content = json.loads(f.read())
        disciplinas.append(content)

In [429]:
disciplinas[1000]

{'unidade': 'Escola Politécnica',
 'departamento': 'Eng Telecomunicações e Controle',
 'campus': 'São Paulo',
 'codigo': 'PTC3502',
 'nome': 'Estágio Supervisionado',
 'creditos_aula': 1,
 'creditos_trabalho': 6,
 'objetivos': 'Realização de um estágio relacionado ao futuro exercício profissional, com a supervisão de um professor do curso.',
 'programa_resumido': 'Estágio em empresa, universidade ou instituição de pesquisa.',
 'turmas': [{'codigo': '2022250',
   'inicio': '15/08/2022',
   'fim': '21/12/2022',
   'tipo': 'Teórica',
   'horario': [{'dia': 'sab',
     'inicio': '09:20',
     'fim': '10:10',
     'professores': ['Cristiano Magalhaes Panazio']}],
   'vagas': {'Obrigatória': {'vagas': 35,
     'inscritos': 8,
     'pendentes': 0,
     'matriculados': 8,
     'grupos': {'EP - Engenharia Elétrica - EC3 (Telecomunicações)': {'vagas': 35,
       'inscritos': 6,
       'pendentes': 0,
       'matriculados': 6}}},
    'Extracurricular': {'vagas': 2,
     'inscritos': 0,
     'pend

##### Filtrar Institutos CUASO

In [430]:
fora_cuaso = ['Escola de Artes, Ciências e Humanidades', 'Escola de Enfermagem', 'Faculdade de Direito', 'Faculdade de Medicina',
              'Faculdade de Saúde Pública', 'Faculdade de Saúde Pública e Faculdade de Ciências Farmacêuticas', 'Escola de Enfermagem de Ribeirão Preto',
              'Museu de Zoologia']
def is_disciplina_cuaso(disciplina, fora_cuaso):
    if disciplina['campus'] != 'São Paulo':
        return False
    if disciplina['unidade'] in fora_cuaso:
        return False
    return True

In [431]:
disciplinas_cuaso = []
for disciplina in disciplinas:
    if is_disciplina_cuaso(disciplina, fora_cuaso):
        disciplinas_cuaso.append(disciplina)
len(disciplinas_cuaso)

2352

In [432]:
unidades_matrusp = {}
for disciplina in disciplinas_cuaso:
    unidades_matrusp[disciplina['unidade']] = 1
unidades_matrusp

{'Instituto Oceanográfico': 1,
 'Escola Politécnica': 1,
 'Escola de Comunicações e Artes': 1,
 'Instituto de Matemática e Estatística': 1,
 'Faculdade de Filosofia, Letras e Ciências Humanas': 1,
 'Instituto de Química': 1,
 'Instituto de Ciências Biomédicas': 1,
 'Instituto de Geociências': 1,
 'Instituto de Biociências': 1,
 'Instituto de Psicologia': 1,
 'Faculdade de Medicina Veterinária e Zootecnia': 1,
 'Faculdade de Ciências Farmacêuticas': 1,
 'Escola de Educação Física e Esporte': 1,
 'Faculdade de Educação': 1,
 'Instituto de Física': 1,
 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas': 1,
 'Faculdade de Odontologia': 1,
 'Faculdade de Arquitetura e Urbanismo': 1,
 'Instituto de Relações Internacionais': 1,
 'Faculdade de Economia, Administração, Contabilidade e Atuária': 1,
 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências': 1,
 'Instituto de Estudos Brasileiros': 1,
 'Museu de Arqueologia e Etnologia': 1

##### Gerar Lista de Aulas

In [433]:
aulas = []
for i, disciplina in enumerate(disciplinas_cuaso):
    for turma in disciplina['turmas']:
        total_matriculados = 0
        for k, vaga in turma['vagas'].items():
            total_matriculados += vaga['matriculados']
            
        if turma['horario'] != None:
            for horarios in turma['horario']:
                aula = {}
                aula['indice_disc'] = i
                aula['codigo'] = disciplina['codigo']
                aula['codigo_turma'] = turma['codigo']
                aula['unidade'] = disciplina['unidade']
                aula['dia'] = horarios['dia']
                aula['horario_inicio'] = horarios['inicio']
                aula['horario_fim'] = horarios['fim']
                aula['total'] = total_matriculados
                aulas.append(aula)

##### Filtrar Aulas por Intervalo Horário

In [434]:
aulas_intervalo_horario = []
for aula in aulas:
    if intervalo_horario(aula['horario_inicio'], aula['horario_fim']):
        aulas_intervalo_horario.append(aula)

# Pesquisa Parte 1

##### Ler Pesquisa

In [435]:
import pandas as pd
import datetime as dt

In [436]:
pesquisa_perguntas = pd.read_csv('./data/disciplinas/forms/resposta_pesq.csv', delimiter=';')
pesquisa_perguntas['timestamp'] = list(map(lambda x: dt.datetime.strptime(x, '%m/%d/%Y %H:%M:%S'), pesquisa_perguntas['Carimbo de data/hora']))
pesquisa_perguntas = pesquisa_perguntas[pesquisa_perguntas['timestamp'] >= dt.datetime(2022, 10, 7, 0, 0, 0)]
len(pesquisa_perguntas)

4395

In [437]:
pesquisa = pd.DataFrame()
pesquisa['vinculo'] = pesquisa_perguntas['Qual o seu vínculo com a USP?']
pesquisa['instituto'] = pesquisa_perguntas['A qual instituto você está associado?']


# ida
pesquisa['utiliza_circular_ida'] = pesquisa_perguntas['Quantas vezes por semana você utiliza o circular para entrar na USP?'] >= 2
pesquisa['linhas_ida'] = pesquisa_perguntas['Qual das linhas abaixo você utiliza neste trajeto?']
pesquisa['ponto_embarque_ida'] = pesquisa_perguntas['Por qual ponto você normalmente embarca?']
pesquisa['ponto_desembarque_ida'] = pesquisa_perguntas['Em qual ponto de ônibus você normalmente desembarca?']
pesquisa['horario_ida'] = pesquisa_perguntas['Que horas você normalmente chega na USP?']


# volta
pesquisa['utiliza_circular_volta'] = pesquisa_perguntas['Quantas vezes por semana você utiliza o circular para sair da USP?'] >= 2
pesquisa['linhas_volta'] = pesquisa_perguntas['Qual das linhas abaixo você utiliza neste trajeto?.1']
pesquisa['ponto_embarque_volta'] = pesquisa_perguntas['Em qual o ponto de ônibus você embarca para sair da USP?']
pesquisa['ponto_desembarque_volta'] = pesquisa_perguntas['Por qual ponto você normalmente desembarca?']
pesquisa['horario_volta'] = pesquisa_perguntas['Que horas você normalmente chega no ponto de ônibus para sair da USP?']

pesquisa.head()

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
136,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8022",P3,FAU II,7:54:00 AM,True,"8012, 8022",FAU II,Portão da Vila Indiana,7:10:00 PM
137,Aluno de graduação,Faculadae de Arquitetura e Urbanismo (FAU),True,"8012, 8032",Estação Butantã,FEA,1:10:00 PM,True,"8012, 8022",FAU II,Estação Butantã,10:30:00 PM
138,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8032",Estação Butantã,FEA,7:15:00 PM,True,8022,Rua do Matão 0,Estação Butantã,10:40:00 PM
139,Aluno de pós-graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,8012,PTREM,Geografia/História,7:25:00 PM,True,8012,Geografia/História,PTREM,9:40:00 PM
140,Aluno de graduação,Instituto de Relações internacionais (IRI),False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN


In [438]:
pesquisa['instituto'].unique()

array(['Instituto de Matemática e Estatística (IME)',
       'Faculadae de Arquitetura e Urbanismo (FAU)',
       'Faculdade de Filosofia, Letras e Ciências Humanas (FFLCH)',
       'Instituto de Relações internacionais (IRI)',
       'Faculdade de Ciências Farmacêuticas (FCF)',
       'Escola Politécnica (Poli)',
       'Faculdade de Economia, Administração e Contabilidade (FEA)',
       'Instituto de Química (IQ)',
       'Escola de Artes, Ciências e Humanidades (EACH)',
       'Instituto de Física (IF)', 'Faculdade de Medicina (FM)',
       'Instituto Oceanográfico (IO)',
       'Escola de Comunicações e Artes (ECA)',
       'Instituto de Pesquisas Energéticas e Nucleares (IPEN)',
       'Instituto de Biociências (IB)', 'Faculdade de Educação (FE)',
       'Instituto de Ciências Biomédicas (ICB)',
       'Faculdade de Odontologia (FO)',
       'Escola de Educação Física e Esporte (EEFE)', 'PRG',
       'Instituo de Astronomia, Geofísica e Ciências Atmosféricas (IAG)',
       'Instit

##### Filtrar Vínculo

In [439]:
pesquisa[['vinculo']].value_counts()

vinculo                                                        
Aluno de graduação                                                 2880
Aluno de pós-graduação                                              752
Funcionário                                                         644
Docente                                                              36
docente                                                              22
Professor                                                            21
professor                                                             8
Ex-aluno                                                              5
Professor                                                             5
Docente                                                               4
Professor titular                                                     1
aprimoramento                                                         1
Pós-doutorando                                                        1


In [440]:
pesquisa[['vinculo', 'utiliza_circular_' + sentido]].value_counts()

vinculo                                                          utiliza_circular_ida
Aluno de graduação                                               True                    2532
Aluno de pós-graduação                                           True                     575
Funcionário                                                      True                     369
Aluno de graduação                                               False                    348
Funcionário                                                      False                    275
Aluno de pós-graduação                                           False                    177
Docente                                                          True                      28
docente                                                          False                     17
Professor                                                        True                      11
                                                                 Fal

In [441]:
pesquisa_vinculo = pesquisa[list(map(filtra_vinculo, pesquisa['vinculo']))]
pesquisa_vinculo[['vinculo']].value_counts()

vinculo    
Funcionário    644
dtype: int64

##### Normaliza instituto com unidade do matrusp ou anuário

In [442]:
def retira_sigla(x):
    index = x.find('(')
    
    if index < 0:
        return x
    
    return x[:index - 1]

def filtra_instituto_pesq(x):
    x = retira_sigla(x)
    
    if x in ['ECA e MAC USP', 'Escola de Artes, Ciências e Humanidades']: 
        return 'Escola de Comunicações e Artes'
    elif x == 'FFLCH':
        return 'Faculdade de Filosofia, Letras e Ciências Humanas'
    elif x in ['IPEN', 'Instituto de Pesquisas Energéticas e Necleares', 'IPEN ', 'ipen', 'Instituto de pesquisas energéticas e nucleares', 'Instituto de Pesquisa Energéticas e Nucleares', 'Instituto de Pesquisas Energéticas e Nucleares - IPEN', 'Instituto de pesquisas energéticas e nucleares ', 'Instituto de pesquisas energéticas e nucleares -IPEN', 'Instituo de Pesquisas Energéticas e Nucleares']:
        return 'Instituto de Pesquisas Energéticas e Nucleares'
    elif x in ['MAC', 'MAC USP']:
        return 'Museu de Arte Contemporânea'
    elif x == 'Faculadae de Arquitetura e Urbanismo':
        return 'Faculdade de Arquitetura e Urbanismo'
    elif x in ['IAG ', 'IAG', 'Instituo de Astronomia, Geofísica e Ciências Atmosféricas']:
        return 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas'
    elif x == 'Faculdade de Economia, Administração e Contabilidade':
        return 'Faculdade de Economia, Administração, Contabilidade e Atuária'
    elif x in ['Escola de Aplicação da Faculdade de Educação (EA-FEUSP)']:
        return 'Escola de Educação Física e Esporte'
    elif x == 'Instituto de Relações internacionais':
        return 'Instituto de Relações Internacionais'
    elif x in ['Museu de arqueologia e etnologia ']:
        return 'Museu de Arqueologia e Etnologia'
    elif x in ['Instituto de Energia e Ambientes']:
        return 'Instituto de Energia e Ambiente'
    elif x in ['Hospital universitário da usp', 'Hospital universitário ', 'Hospital HU', 'Hosp. Universitario', 'HU-USP', 'Hospital Universitário da USP ', 'HOSPITAL UNIVERSITÁRIO', 'HOSPITAL UNIVERSITÁRIO - HU', 'Hospital Universitário da USP', 'HU USP', 'hospital universitário', 'HU', 'Hospital Universitário HU', 'Hu', 'HOSPITAL UNIVERSITARIO', 'Hospital universitário', 'Hospital  Universitário ', 'hu', 'HU ', 'Hospital Universitário ']:
        return 'Hospital Universitário'
    elif x in ['SAS', 'sas', 'Superintendência de Comunicação Social', 'Prip', 'edusp', 'pusp c', 'PUSP-C','pusp-c', 'Prédio da Reitoria', 'Prédio da Administração Central-SEF', 'Prefeitura do Campus USP da Capital', 'SUB REITORIA', 'Reitoria.', 'Prip usp', 'Pró-Reitoria de Inclusão e Pertencimento - PRIP', 'pro reitoria', 'Pro reitoria de cultura e extensão ', 'PRIP/Reitoria', 'Edusp', 'PUSP C', 'PUS-C', 'Pusp', 'Prefeitura do Campos', 'Prefeitura ', 'Reitoria da USP', 'Editora da Universidade de São Paulo - Edusp', 'Pusp-c', 'PRG', 'Prg ', 'Pró-Reitoria de Graduação', 'Pró-Reitoria de graduação', 'Pró-reitoria de graduacao', 'Reitoria', 'Reitoria usp', 'REITORIA', 'Pró-Reitoria de Inclusão e Pertencimento - PRIP-USP', 'Pró-Reitoria de Inclusão e Pertencimento', 'Reitoria da Universidade de São Paulo', 'Reitoria - Administração Central', 'Reitoria ', 'rusp', 'RUSP', 'Editora da USP']:
        return 'Reitoria Administração Central'
    elif x in ['Museu de Zoologia', 'Museu de Zoologia da USP']:
        return 'Faculdade de Medicina Veterinária e Zootecnia'
    elif x in ['Superintendencia de Tecnologia da Informação', 'superintendencia do espaço fisico', 'Superintendência de Segurança - SPPU', 'SUperintendencia de Tecnologia e INformação', 'STI - CeTI-SP']:
        return 'Superintendência da Tecnologia da Informação'
    elif x in ['Cepeusp ', 'Cepeusp', 'Centro de Práticas Esportivas da USP - CEPEUSP', 'CEPEUSP']:
        return 'Centro de Práticas Esportivas'
    elif x in ['Curso de Ciências Moleculares', 'Prceu ', 'PRCEU', 'Clinica Odontológica da SAU no CRUSP', 'PRIP', 'STI', 'Superintendência de Tecnologia da Informação - STI', 'CEPEUSP ', 'SEF', 'Coseas', 'Superintendência do Espaço Físico', 'BBM-USP', 'CeTISP', 'ABCD', 'Restaurante universitario central', 'Restaurante Central ', 'Aucani', 'Centro de Saúde Escola Samuel Barsley Pessoa ', 'Osusp', 'SCS', 'Museu Paulista', 'Museu de Arqueologia e Etnologia ', 'CSCRH-RP-01', 'Instituto de Medicina Tropical de São Paulo', 'Rádio USP ', 'Perceu', 'USP Centro de Tecnologia de Informação - CETISP']:
        None
    else:
        return x
    
pesquisa_vinculo2 = pesquisa_vinculo.copy()
pesquisa_vinculo2['instituto'] = list(map(filtra_instituto_pesq, pesquisa_vinculo['instituto']))

In [443]:
# Filtra os Nones

pesquisa_vinculo = pesquisa_vinculo2
pesquisa_vinculo = pesquisa_vinculo[pesquisa_vinculo['instituto'].notna()]
pesquisa_vinculo['instituto'].unique()

array(['Instituto de Matemática e Estatística', 'Instituto de Física',
       'Hospital Universitário', 'Reitoria Administração Central',
       'Faculdade de Medicina', 'Faculdade de Educação',
       'Faculdade de Medicina Veterinária e Zootecnia',
       'Faculdade de Odontologia', 'Escola de Comunicações e Artes',
       'Faculdade de Ciências Farmacêuticas',
       'Faculdade de Filosofia, Letras e Ciências Humanas',
       'Escola Politécnica', 'Instituto de Geociências',
       'Instituto de Ciências Biomédicas', 'Instituto de Química',
       'Escola de Educação Física e Esporte',
       'Museu de Arte Contemporânea',
       'Faculdade de Economia, Administração, Contabilidade e Atuária',
       'Instituto Oceanográfico', 'Instituto de Energia e Ambiente',
       'Instituto de Biociências', 'Escola de Enfermagem',
       'Hospital Universitario', 'Faculdade de Arquitetura e Urbanismo',
       'Instituto de Astronomia, Geofísica e Ciências Atmosféricas',
       'Instituto de Rel

##### Mapeia Institutos Fora CUASO

In [444]:
def mapeia_cursos_de_fora(x):
    if x in ['Faculdade de Medicina', 'Escola de Enfermagem', 'Faculdade de Saúde Pública']:
        return 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências'
    elif x == 'Faculdade de Direito':
        return 'Faculdade de Filosofia, Letras e Ciências Humanas'
    elif x == 'MAC':
        return 'Museu de Arte Contemporânea'
    else:
        return x
    
pesquisa_vinculo2 = pesquisa_vinculo.copy()
pesquisa_vinculo2['instituto'] = list(map(mapeia_cursos_de_fora, pesquisa_vinculo['instituto']))
pesquisa_vinculo = pesquisa_vinculo2

In [445]:
pesquisa_vinculo['instituto'].unique()

array(['Instituto de Matemática e Estatística', 'Instituto de Física',
       'Hospital Universitário', 'Reitoria Administração Central',
       'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências',
       'Faculdade de Educação',
       'Faculdade de Medicina Veterinária e Zootecnia',
       'Faculdade de Odontologia', 'Escola de Comunicações e Artes',
       'Faculdade de Ciências Farmacêuticas',
       'Faculdade de Filosofia, Letras e Ciências Humanas',
       'Escola Politécnica', 'Instituto de Geociências',
       'Instituto de Ciências Biomédicas', 'Instituto de Química',
       'Escola de Educação Física e Esporte',
       'Museu de Arte Contemporânea',
       'Faculdade de Economia, Administração, Contabilidade e Atuária',
       'Instituto Oceanográfico', 'Instituto de Energia e Ambiente',
       'Instituto de Biociências', 'Hospital Universitario',
       'Faculdade de Arquitetura e Urbanismo',
       'Instituto de Astron

##### Filtrar respostas pesquisa pelo Intervalo Horário

In [446]:
pesquisa_vinculo.head()

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
249,Funcionário,Instituto de Matemática e Estatística,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
282,Funcionário,Instituto de Física,False,"8012, 8022, 8032",P1,Poli Biênio,7:30:00 AM,True,8022,Poli Biênio,Estação Butantã,5:00:00 PM
523,Funcionário,Hospital Universitário,True,"8012, 8032",Estação Butantã,Hospital Universitário,7:00:00 AM,True,8032,Hospital Universitário,Estação Butantã,7:05:00 PM
601,Funcionário,Reitoria Administração Central,True,"8012, 8032",Estação Butantã,Reitoria/Bancos,9:05:00 AM,True,"8012, 8032",Psicologia II,Estação Butantã,6:10:00 PM
844,Funcionário,"Faculdade de Medicina, Instituto de Ciências B...",True,8012,Estação Butantã,P3,9:00:00 AM,True,8022,P3,Estação Butantã,6:00:00 PM


In [447]:
pesquisa_vinculo['no_horario'] = list(map(intervalo_horario_am_pm, pesquisa_vinculo['horario_' + sentido]))
pesquisa_vinculo_intervalo = pesquisa_vinculo[pesquisa_vinculo['no_horario']]
pesquisa_vinculo_intervalo

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta,no_horario
282,Funcionário,Instituto de Física,False,"8012, 8022, 8032",P1,Poli Biênio,7:30:00 AM,True,8022,Poli Biênio,Estação Butantã,5:00:00 PM,True
523,Funcionário,Hospital Universitário,True,"8012, 8032",Estação Butantã,Hospital Universitário,7:00:00 AM,True,8032,Hospital Universitário,Estação Butantã,7:05:00 PM,True
972,Funcionário,Faculdade de Educação,True,8022,Portão da Vila Indiana,Avenida da Universidade,7:05:00 AM,False,"8012, 8022",Avenida da Universidade,Estação Butantã,2:15:00 PM,True
1381,Funcionário,Faculdade de Ciências Farmacêuticas,True,8022,P3,Portão da Vila Indiana,7:00:00 AM,True,8022,Farmácia e Química,P3,4:15:00 PM,True
1424,Funcionário,"Faculdade de Filosofia, Letras e Ciências Humanas",True,"8012, 8022, 8032",Portaria São Remo,Geografia/História,7:00:00 AM,True,8012,Letras,Portaria São Remo,5:00:00 AM,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4442,Funcionário,Instituto de Geociências,True,"8012, 8022",P3,FEA,8:00:00 AM,True,"8012, 8022",FAU II,Estação Butantã,5:10:00 PM,True
4444,Funcionário,Instituto de Geociências,True,"8012, 8022, 8032",Estação Butantã,Reitoria/Bancos,8:15:00 AM,True,"8012, 8022, 8032",Estação Butantã,Estação Butantã,5:00:00 PM,True
4447,Funcionário,Reitoria Administração Central,True,"8012, 8032",Estação Butantã,Estação Butantã,8:20:00 AM,True,"8012, 8032","R. Prof. Mello Moraes, 1473 (localizado entre ...",Estação Butantã,5:20:00 AM,True
4450,Funcionário,Escola Politécnica,True,8032,Estação Butantã,Poli Biênio,8:15:00 AM,True,8032,Poli Biênio,Estação Butantã,5:05:00 PM,True


# Funcionários

##### Ler os dados do anuário

In [448]:
with open('data/funcionarios_anuario.json', 'r+') as f:
    str_funcionarios_anuario = f.read()
    funcionarios_anuario = json.loads(str_funcionarios_anuario)
funcionarios_anuario[0]

{'unidade': 'Faculdade de Arquitetura e Urbanismo', 'funcionarios': 129}

##### Filtrar Horário baseado na pesquisa

In [449]:
no_horario_por_instituto = pesquisa_vinculo[['instituto', 'no_horario']].groupby('instituto').agg('mean').reset_index()
no_horario_por_instituto[no_horario_por_instituto['instituto'] == 'Instituto de Matemática e Estatística']

,instituto,no_horario
23,Instituto de Matemática e Estatística,0.375


In [450]:
funcionarios_intervalo_horario = []
if vinculo == 'funcionario':
    for funcionario_anuario in funcionarios_anuario:
        unidade = funcionario_anuario['unidade']
        porc_no_horario = no_horario_por_instituto[no_horario_por_instituto['instituto'] == unidade]['no_horario'].iloc[0]
        for dia in dias:
            funcionarios_intervalo_horario.append({
                'unidade': unidade,
                'dia': dia,
                'total': funcionario_anuario['funcionarios'] * porc_no_horario,
            })
    funcionarios_intervalo_horario[0]

# Pesquisa Parte 2

In [451]:
pesquisa_vinculo_intervalo['ponto_embarque_ida'].unique()

array(['P1', 'Estação Butantã', 'Portão da Vila Indiana', 'P3',
       'Portaria São Remo', 'PTREM', 'P2',
       'R. Prof. Mello Moraes, 900 (localizado entre a CRUSP e a Raia, sentindo P3)',
       'R. Prof. Mello Moraes, 1473 (localizado entre a CRUSP e a Raia, sentindo Metrô Butantã)',
       'Biblioteca Brasiliana'], dtype=object)

##### Mapeia Pontos Pesquisa para ficar igual ao da API

In [452]:
import requests
pontos = requests.get('https://uspdigital.usp.br/mobile/servicos/mapa/locais?campus=cidade-universitaria&categorias=transportes')
pontos = json.loads(pontos.text)
pontos[10]

{'categoria': 'transportes',
 'campus': 'cidade-universitaria',
 'id': '1811',
 'titulo': 'Ponto de ônibus ECA',
 'latitude': '-23.557806338669685',
 'longitude': '-46.72689100000002',
 'endereco': 'Av. Prof. Lúcio Martins Rodrigues',
 'imagem': 'http://www.usp.br/mapas/wp-content/uploads/10042014giroculturalfotomarcossantos008.jpg',
 'url': '',
 'descricao': 'Atende as linhas  701U-10, 702U-10, 7725-10, 8012-10'}

In [453]:
nomes_pontos_req = []
for ponto in pontos:
    if ponto['campus'] == 'cidade-universitaria':
        nomes_pontos_req.append(ponto['titulo'])
 
nomes_pontos_pesq = []
for ponto in pesquisa_vinculo_intervalo['ponto_' + acao + '_' + sentido].unique():
    nomes_pontos_pesq.append(ponto)

print(nomes_pontos_req)
print(nomes_pontos_pesq)

['Ponto de ônibus metrô Butantã', 'Ponto de ônibus Afrânio Peixoto', 'Ponto de ônibus Poli Metalúrgica', 'Terminal de ônibus USP', 'Ponto de ônibus Poli Mecânica', 'Ponto de ônibus portaria II', 'Ponto de ônibus Hidráulica', 'Ponto de Psicologia I', 'Ponto de ônibus Barracões', 'Ponto de ônibus ECA', 'Ponto de ônibus Praça do Relógio', 'Ponto de ônibus - Praça do Relógio', 'Ponto de ônibus Psicologia II', 'Ponto de ônibus - Acesso CPTM II', 'Ponto de ônibus - Acesso CPTM I', 'Ponto de ônibus Escola de Educação Física II', 'Ponto de ônibus Escola de Educação Física I', 'Ponto de ônibus Academia de Polícia', 'Ponto de ônibus Paços das Artes', 'Ponto de ônibus Educação', 'Ponto de ônibus CRUSP', 'Ponto de ônibus Cultura Japonesa', 'Ponto de ônibus Biblioteca Brasiliana', 'Ponto de ônibus Letras', 'Ponto de ônibus Geociência', 'Ponto de ônibus Reitoria/Bancos', 'Ponto de ônibus FEA', 'Ponto de ônibus FAU II', 'Ponto de ônibus Poli Biênio', 'Ponto de ônibus Poli Eletrotécnica', 'Ponto de ôn

In [454]:
def iguais_com_prefixo(x):
    aux_nomes_pontos_pesq = ['Ponto de ônibus ' + x for x in nomes_pontos_pesq]
    if 'Ponto de ônibus ' + x in [item for item in aux_nomes_pontos_pesq if item in list(set(nomes_pontos_req) & set(aux_nomes_pontos_pesq))]:
        return 'Ponto de ônibus ' + x
    else:
        return x
 
def pesquisa_to_pontos_req(x):
    if x in ["Geografia/História", "História e Geografia", "Químicas"]:
        return "Ponto e ônibus História e Geografia"
    elif x  == "Poli Metalurgia":
        return "Ponto de ônibus Poli Metalúrgica"
    elif x == "P3":
        return "Ponto de ônibus Portaria III"
    elif x == "Farmácia e Química":
        return "Ponto de ônibus Biblioteca Farmácia e Química"
    elif x in ["CRUSP (Praça da Reitoria)", "R. Prof. Mello Moraes, 1473 (localizado entre a CRUSP e a Raia, sentindo Metrô Butantã)", "R. Prof. Mello Moraes, 900 (localizado entre a CRUSP e a Raia, sentindo P3)"]:
        return "Ponto de ônibus CRUSP"
    elif x == "Biomédicas":
        return "Ponto de ônibus Biomédicas III"
    elif x in ["Prefeitura/Física", "Ponto de ônibus Ponto Clube dos funcionários"]:
        return "Ponto Clube dos funcionários"
    elif x == "Educação Física":
        return "Ponto de ônibus Escola de Educação Física I"
    elif x in ["Estação Butantã", "Terminal Cidade Universitária"]:
        return "Ponto de ônibus metrô Butantã"
    elif x == "Poli-Eletrotécnia":
        return "Ponto de ônibus Poli Eletrotécnica"
    elif x == "Geociências":
        return "Ponto de ônibus Geociências I"
    elif x in ["icb ", "Biociências", "ICB 1 (Av. Prof. Lineu Prestes, 1524)"]:
        return "Ponto de ônibus  Parada Matemática"
    elif x in ["FE", "Avenida da Universidade", "Faculdade de Educação ", "Educação / CEPEUSP", " FE", "Prédio da FE", "Educação ", "Educação", "Faculdade de Educação"]:
        return "Ponto de ônibus Educação"
    elif x in ["Rua do matão frente da FAU", "Instituto Oceanografico", "Oceanográfico, em frent ao da FAU II", "Rua do Matão, 0 (em frente a FAU)", "Estacionamento FAU", "R. do Matão, frente à FAU", "R. do Matão 878", "R. do Matão, 0", "Oceanográfico", "Oceanografia", "Rua do Matao ", "Matão, Oceanografico", "Rua do Matão, 0 - FAU", "IO USP", "IO", "Rua do matao, 0", "Rua do Matão 0 (FAU)", "Rua do Matão (na frente da FAU)", "Intuito Oceanografico", "R. do Matão", "Oceanografia ", "Oceanógrafico/Matão", "Rua do Matão, 0", "R. Do matão ",  "Rua do Matão", "Ponto em frente à FAU na Rua do Matão"]:
        return "Ponto de ônibus FAU I"
    elif x in ["depois da biblioteca da química ", "Rua do Lago", "Lineu Prestes entre a biblioteca da quimica e o icb", "Ponto do Lago", "Av. Prof. Lineu Prestes, 0"]:
        return "Ponto de ônibus rua do Lago"
    elif x == "Psicologia I":
        return "Ponto de ônibus Psicologia II"
    elif x in ["Cepam",  "Biomédicas, Biologia e Quimica"]:
        return "Ponto de ônibus CEPAM"
    elif x in ["Ipen", "FO ou Ipen", "Marinha/ Odontologia"]:
        return "Ponto de ônibus IPEN"
    elif x in ["Acesso Vila Indiana", "Portão da Vila Indiana"]:
        return "Ponto de ônibus acesso Vl. Indiana"
    elif x == "PTREM":
        return "Ponto de ônibus - Acesso CPTM I"
    elif x == "IPT":
        return "Ponto de ônibus Cocesp I"
    elif x in ["Centro De Difusão Internacional", "Poli Hidráulica"]:
        return "Ponto de ônibus Hidráulica"
    elif x == "Avenida Professor Luciano Gualberto (depois da Parada I.P.T, no 8022)":
        return "Ponto de ônibus Poli Eletrotécnica"
    elif x in ["P1", "Instituto Butantan ", "Academia de polícia"]:
        return "Ponto de ônibus Academia de Polícia"
    elif x == "P2":
        return "Ponto de ônibus portaria II"
    elif x in ["FAU ", "FEA ou FAU II", "As vezes desembarco na FAU II ou na FEA"]:
        return "Ponto de ônibus FAU II"
    elif x == "Portaria São Remo":
        return "Ponto de ônibus acesso Rio Pequeno"
    elif x == "If e Ipen":
        return "Ponto de ônibus Física"
    elif x == "Casa de Cultura Japonesa, ETEC CEPAM":
        return "Ponto de ônibus Cultura Japonesa"
    elif x == "IEB":
        return "Ponto de ônibus Biblioteca Brasiliana"
    elif x in ["IAG (ou, às vezes, Física)", "IAG ou Física", "IAG (sentido Metrô Butantã)"]:
        return "Ponto de ônibus IAG"
    elif x == "Prof Almeida Prado":
        return "Terminal de ônibus USP"
    elif x == "HU":
        return "Ponto de ônibus da COPESP"
    elif x == "pço das artes":
        return "Ponto de ônibus Paços das Artes"
    elif x == "Depende do dia":
        return None
    else:
        return x

In [455]:
str_pontos = ['ponto_desembarque_ida', 'ponto_embarque_ida', 'ponto_desembarque_volta', 'ponto_embarque_volta']
for str_ponto in str_pontos:
    pesquisa_vinculo_intervalo[str_ponto] = list(map(lambda x: pesquisa_to_pontos_req(iguais_com_prefixo(x)), pesquisa_vinculo_intervalo[str_ponto]))
    pesquisa_vinculo_intervalo = pesquisa_vinculo_intervalo[pesquisa_vinculo_intervalo[str_ponto].notna()]

    for ponto in pesquisa_vinculo_intervalo[str_ponto].unique():
        if ponto not in nomes_pontos_req:
            print(ponto)

IEE
Início da rua do Matão, próximo ao ICB
Reitoria - Antiga, Ponto dos Restaurantes 
IME
Rua do Matão (acesso FAU)
MAE
COPESP
Rua do Matão (acesso FAU)
rua do matao


/var/folders/wt/sljrl31144s8xk7szzq6n62h0000gn/T/com.apple.shortcuts.mac-helper/ipykernel_16098/2848880336.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pesquisa_vinculo_intervalo[str_ponto] = list(map(lambda x: pesquisa_to_pontos_req(iguais_com_prefixo(x)), pesquisa_vinculo_intervalo[str_ponto]))


##### Filtrar Terminal

In [456]:
pesquisa_vinculo_intervalo

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta,no_horario
282,Funcionário,Instituto de Física,False,"8012, 8022, 8032",Ponto de ônibus Academia de Polícia,Ponto de ônibus Poli Biênio,7:30:00 AM,True,8022,Ponto de ônibus Poli Biênio,Ponto de ônibus metrô Butantã,5:00:00 PM,True
523,Funcionário,Hospital Universitário,True,"8012, 8032",Ponto de ônibus metrô Butantã,Ponto de ônibus Hospital Universitário,7:00:00 AM,True,8032,Ponto de ônibus Hospital Universitário,Ponto de ônibus metrô Butantã,7:05:00 PM,True
972,Funcionário,Faculdade de Educação,True,8022,Ponto de ônibus acesso Vl. Indiana,Ponto de ônibus Educação,7:05:00 AM,False,"8012, 8022",Ponto de ônibus Educação,Ponto de ônibus metrô Butantã,2:15:00 PM,True
1381,Funcionário,Faculdade de Ciências Farmacêuticas,True,8022,Ponto de ônibus Portaria III,Ponto de ônibus acesso Vl. Indiana,7:00:00 AM,True,8022,Ponto de ônibus Biblioteca Farmácia e Química,Ponto de ônibus Portaria III,4:15:00 PM,True
1424,Funcionário,"Faculdade de Filosofia, Letras e Ciências Humanas",True,"8012, 8022, 8032",Ponto de ônibus acesso Rio Pequeno,Ponto e ônibus História e Geografia,7:00:00 AM,True,8012,Ponto de ônibus Letras,Ponto de ônibus acesso Rio Pequeno,5:00:00 AM,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4442,Funcionário,Instituto de Geociências,True,"8012, 8022",Ponto de ônibus Portaria III,Ponto de ônibus FEA,8:00:00 AM,True,"8012, 8022",Ponto de ônibus FAU II,Ponto de ônibus metrô Butantã,5:10:00 PM,True
4444,Funcionário,Instituto de Geociências,True,"8012, 8022, 8032",Ponto de ônibus metrô Butantã,Ponto de ônibus Reitoria/Bancos,8:15:00 AM,True,"8012, 8022, 8032",Ponto de ônibus metrô Butantã,Ponto de ônibus metrô Butantã,5:00:00 PM,True
4447,Funcionário,Reitoria Administração Central,True,"8012, 8032",Ponto de ônibus metrô Butantã,Ponto de ônibus metrô Butantã,8:20:00 AM,True,"8012, 8032",Ponto de ônibus CRUSP,Ponto de ônibus metrô Butantã,5:20:00 AM,True
4450,Funcionário,Escola Politécnica,True,8032,Ponto de ônibus metrô Butantã,Ponto de ônibus Poli Biênio,8:15:00 AM,True,8032,Ponto de ônibus Poli Biênio,Ponto de ônibus metrô Butantã,5:05:00 PM,True


In [457]:
pesquisa_vinculo_intervalo_terminal = pesquisa_vinculo_intervalo[pesquisa_vinculo_intervalo['utiliza_circular_' + sentido]]
pesquisa_vinculo_intervalo_terminal = pesquisa_vinculo_intervalo_terminal[list(map(filtra_terminal, pesquisa_vinculo_intervalo_terminal['ponto_' + contrario_acao + '_' + sentido]))]
pesquisa_vinculo_intervalo_terminal

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta,no_horario
972,Funcionário,Faculdade de Educação,True,8022,Ponto de ônibus acesso Vl. Indiana,Ponto de ônibus Educação,7:05:00 AM,False,"8012, 8022",Ponto de ônibus Educação,Ponto de ônibus metrô Butantã,2:15:00 PM,True
1381,Funcionário,Faculdade de Ciências Farmacêuticas,True,8022,Ponto de ônibus Portaria III,Ponto de ônibus acesso Vl. Indiana,7:00:00 AM,True,8022,Ponto de ônibus Biblioteca Farmácia e Química,Ponto de ônibus Portaria III,4:15:00 PM,True
1781,Funcionário,Instituto de Psicologia,True,"8022, 8032",Ponto de ônibus Portaria III,Ponto de ônibus Psicologia II,7:30:00 AM,True,"8012, 8022",Ponto de ônibus Psicologia II,Ponto de ônibus CRUSP,8:00:00 AM,True
1929,Funcionário,Reitoria Administração Central,True,8012,Ponto de ônibus Portaria III,Ponto de ônibus ECA,7:20:00 AM,True,8012,Ponto de ônibus Reitoria/Bancos,Ponto de ônibus Portaria III,7:00:00 AM,True
2075,Funcionário,Hospital Universitário,True,"8012, 8022",Ponto de ônibus Portaria III,Ponto de ônibus Academia de Polícia,7:30:00 AM,True,"8012, 8022",Ponto de ônibus Portaria III,Ponto de ônibus Academia de Polícia,6:50:00 AM,True
2103,Funcionário,"Instituto de Astronomia, Geofísica e Ciências ...",True,"8012, 8022",Ponto de ônibus Portaria III,Ponto de ônibus Física,8:30:00 AM,False,8022,Ponto de ônibus Física,Ponto de ônibus acesso Vl. Indiana,5:00:00 PM,True
2590,Funcionário,Superintendência da Tecnologia da Informação,True,"8012, 8022",Ponto de ônibus acesso Vl. Indiana,Ponto de ônibus Poli Civil,8:30:00 AM,True,8012,Ponto de ônibus Butantan,Ponto de ônibus acesso Vl. Indiana,5:30:00 PM,True
2749,Funcionário,Instituto de Geociências,True,"8012, 8022",Ponto de ônibus Portaria III,Ponto de ônibus FAU II,8:30:00 AM,True,8022,Ponto de ônibus FAU II,Ponto de ônibus Portaria III,4:45:00 PM,True
2782,Funcionário,Reitoria Administração Central,True,8012,Ponto de ônibus Portaria III,Ponto de ônibus CRUSP,7:20:00 AM,True,8012,Ponto de ônibus Biblioteca Brasiliana,Ponto de ônibus acesso Rio Pequeno,4:30:00 PM,True
3396,Funcionário,Instituto de Psicologia,True,"8012, 8022",Ponto de ônibus Portaria III,Ponto de ônibus Psicologia II,6:30:00 AM,True,8022,Ponto de ônibus Psicologia II,Ponto de ônibus Portaria III,4:45:00 PM,True


##### Calcular Porcentagem Carro por Instituto

In [458]:
porc_circular_por_instituto = pesquisa_vinculo_intervalo_terminal[['instituto','utiliza_circular_' + sentido]].groupby('instituto').agg('mean').reset_index()
porc_circular_por_instituto

,instituto,utiliza_circular_ida
0,Escola Politécnica,1.0
1,Escola de Comunicações e Artes,1.0
2,Faculdade de Arquitetura e Urbanismo,1.0
3,Faculdade de Ciências Farmacêuticas,1.0
4,Faculdade de Educação,1.0
5,"Faculdade de Filosofia, Letras e Ciências Humanas",1.0
6,Hospital Universitário,1.0
7,Instituto Oceanográfico,1.0
8,"Instituto de Astronomia, Geofísica e Ciências ...",1.0
9,Instituto de Biociências,1.0


In [459]:
def porc_utiliza_circular(instituto):
    return porc_circular_por_instituto[porc_circular_por_instituto['instituto'] == instituto]['utiliza_circular_' + sentido].iloc[0]

##### Calcula porcentagem para Ponto e Linha por Instituto (filtrando quem usa carro)

In [460]:
import random
def usa_linha(str_linhas, linha_target):
    linhas = str_linhas.split(', ')
    linha_selecionada = linhas[random.randint(0, len(linhas) - 1)]
    return (linha_selecionada == linha_target)

def calcula_quantidade_pessoas_ponto(instituto, ponto, linha):
    df_instituto = pesquisa_vinculo_intervalo_terminal[pesquisa_vinculo_intervalo_terminal['instituto'] == instituto]

    df_linha = df_instituto[list(map(lambda x: usa_linha(x, linha), df_instituto['linhas_' + sentido]))]    
    if len(df_linha['linhas_' + sentido]) == 0:
        return 0

    df_ponto = df_linha[df_linha['ponto_' + acao + '_' + sentido] == ponto]
    if len(df_ponto['ponto_' + acao + '_' + sentido]) == 0:
        return 0

    return df_ponto['ponto_' + acao + '_' + sentido].value_counts().iloc[0]

In [461]:
pontos_existentes = pesquisa_vinculo_intervalo_terminal['ponto_' + acao + '_' + sentido].unique()

institutos_ponto_rel = {}
for instituto in pesquisa_vinculo_intervalo_terminal['instituto'].unique():
    institutos_ponto_rel[instituto] = {}
    soma = len(pesquisa_vinculo_intervalo_terminal[pesquisa_vinculo_intervalo_terminal['instituto'] == instituto]['instituto'])
    for ponto in pontos_existentes:
        for linha in ['8012', '8022', '8032']:
            pessoas = calcula_quantidade_pessoas_ponto(instituto, ponto, linha) * porc_utiliza_circular(instituto)
            if pessoas > 0:
                if ponto not in institutos_ponto_rel[instituto]:
                    institutos_ponto_rel[instituto][ponto] = {}
                institutos_ponto_rel[instituto][ponto][linha] = pessoas / soma
institutos_ponto_rel

{'Faculdade de Educação': {'Ponto de ônibus Educação': {'8022': 1.0}},
 'Faculdade de Ciências Farmacêuticas': {'Ponto de ônibus acesso Vl. Indiana': {'8022': 0.5},
  'Ponto de ônibus Portaria III': {'8022': 0.5}},
 'Instituto de Psicologia': {'Ponto de ônibus Psicologia II': {'8012': 0.3333333333333333,
   '8022': 0.6666666666666666},
  'Ponto de ônibus Reitoria/Bancos': {'8012': 0.3333333333333333}},
 'Reitoria Administração Central': {'Ponto de ônibus ECA': {'8012': 0.16666666666666666},
  'Ponto de ônibus CRUSP': {'8012': 0.16666666666666666,
   '8022': 0.16666666666666666},
  'Ponto de ônibus Reitoria/Bancos': {'8012': 0.16666666666666666},
  'Ponto de ônibus Cultura Japonesa': {'8022': 0.16666666666666666},
  'Ponto de ônibus Hidráulica': {'8012': 0.16666666666666666,
   '8032': 0.16666666666666666}},
 'Hospital Universitário': {},
 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas': {},
 'Superintendência da Tecnologia da Informação': {'Ponto de ônibus Poli Civil': {'8

# Pesquisa e (Aluno/DadosMatrusp ou Funcionário)

In [462]:
aulas_intervalo_horario = get_dados_vinculo(aulas_intervalo_horario, funcionarios_intervalo_horario)

##### Verificar unidades Matrusp e Pesquisa

In [463]:
# Alterar a unidade de aulas na qual a unidade não respondeu a pesquisa

for aula in aulas_intervalo_horario:
    if aula['unidade'] == 'Instituto de Ciências Biomédicas':
        aula['unidade'] = 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências'
    if aula['unidade'] == 'Instituto de Estudos Brasileiros':
        aula['unidade'] = 'Faculdade de Economia, Administração, Contabilidade e Atuária'
    if aula['unidade'] == 'Instituto de Psicologia':
        aula['unidade'] = 'Escola de Comunicações e Artes'

In [464]:
institutos_matrusp = {}
for aula in aulas_intervalo_horario:
    institutos_matrusp[aula['unidade']] = 1
institutos_matrusp = list(institutos_matrusp.keys())
institutos_matrusp.sort()
institutos_matrusp

['Centro de Práticas Esportivas',
 'Escola Politécnica',
 'Escola de Comunicações e Artes',
 'Faculdade de Arquitetura e Urbanismo',
 'Faculdade de Ciências Farmacêuticas',
 'Faculdade de Economia, Administração, Contabilidade e Atuária',
 'Faculdade de Educação',
 'Faculdade de Filosofia, Letras e Ciências Humanas',
 'Faculdade de Medicina Veterinária e Zootecnia',
 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências',
 'Faculdade de Odontologia',
 'Hospital Universitário',
 'Instituto Oceanográfico',
 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas',
 'Instituto de Biociências',
 'Instituto de Física',
 'Instituto de Geociências',
 'Instituto de Matemática e Estatística',
 'Instituto de Pesquisas Energéticas e Nucleares',
 'Instituto de Química',
 'Instituto de Relações Internacionais',
 'Museu de Arte Contemporânea',
 'Reitoria Administração Central',
 'Superintendência da Tecnologia da Informação']

In [465]:
institutos_pesquisa = list(institutos_ponto_rel.keys())
institutos_pesquisa.sort()
institutos_pesquisa

['Escola Politécnica',
 'Escola de Comunicações e Artes',
 'Faculdade de Arquitetura e Urbanismo',
 'Faculdade de Ciências Farmacêuticas',
 'Faculdade de Educação',
 'Faculdade de Filosofia, Letras e Ciências Humanas',
 'Hospital Universitário',
 'Instituto Oceanográfico',
 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas',
 'Instituto de Biociências',
 'Instituto de Ciências Biomédicas',
 'Instituto de Estudos Avançados',
 'Instituto de Geociências',
 'Instituto de Matemática e Estatística',
 'Instituto de Psicologia',
 'Reitoria Administração Central',
 'Superintendência da Tecnologia da Informação']

In [466]:
for instituto_matrusp in institutos_matrusp:
    if instituto_matrusp not in institutos_pesquisa:
        print(instituto_matrusp + ' from matrusp is not in institutos pesquisa')

Centro de Práticas Esportivas from matrusp is not in institutos pesquisa
Faculdade de Economia, Administração, Contabilidade e Atuária from matrusp is not in institutos pesquisa
Faculdade de Medicina Veterinária e Zootecnia from matrusp is not in institutos pesquisa
Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências from matrusp is not in institutos pesquisa
Faculdade de Odontologia from matrusp is not in institutos pesquisa
Instituto de Física from matrusp is not in institutos pesquisa
Instituto de Pesquisas Energéticas e Nucleares from matrusp is not in institutos pesquisa
Instituto de Química from matrusp is not in institutos pesquisa
Instituto de Relações Internacionais from matrusp is not in institutos pesquisa
Museu de Arte Contemporânea from matrusp is not in institutos pesquisa


##### Calcula Map: dia->hora->ponto->linha->pessoas

In [467]:
dia_hora_ponto_linha = {}

def sum_valor_dia_hora_ponto_linha(dia, hora, ponto, linha, valor):
    if dia not in dia_hora_ponto_linha:
        dia_hora_ponto_linha[dia] = {}
    if hora not in dia_hora_ponto_linha[dia]:
        dia_hora_ponto_linha[dia][hora] = {}
    if ponto not in dia_hora_ponto_linha[dia][hora]:
        dia_hora_ponto_linha[dia][hora][ponto] = {}
    if linha not in dia_hora_ponto_linha[dia][hora][ponto]:
        dia_hora_ponto_linha[dia][hora][ponto][linha] = 0
    dia_hora_ponto_linha[dia][hora][ponto][linha] += valor

for aula in aulas_intervalo_horario:
    if aula['unidade'] in institutos_ponto_rel:
        porcentagem_pontos_por_instituto = institutos_ponto_rel[aula['unidade']]
        for ponto in porcentagem_pontos_por_instituto:
            linhas = porcentagem_pontos_por_instituto[ponto]
            for linha in linhas:
                porc = linhas[linha]
                valor = aula['total'] * porc
                sum_valor_dia_hora_ponto_linha(aula['dia'], hora, ponto, linha, valor)

# arredondando
for dia in dia_hora_ponto_linha:
    for hora in dia_hora_ponto_linha[dia]:
        for ponto in dia_hora_ponto_linha[dia][hora]:
            for linha in dia_hora_ponto_linha[dia][hora][ponto]:
                dia_hora_ponto_linha[dia][hora][ponto][linha] = round(dia_hora_ponto_linha[dia][hora][ponto][linha])

dia_hora_ponto_linha

{'seg': {480: {'Ponto de ônibus Portaria III': {'8012': 168, '8022': 123},
   'Ponto de ônibus FAU II': {'8012': 90, '8022': 77},
   'Ponto de ônibus acesso Vl. Indiana': {'8022': 34},
   'Ponto de ônibus ECA': {'8012': 133},
   'Ponto de ônibus metrô Butantã': {'8012': 28, '8032': 28},
   'Ponto de ônibus Cocesp I': {'8012': 128},
   'Ponto de ônibus Poli Metalúrgica': {'8012': 128},
   'Ponto de ônibus Educação': {'8022': 28},
   'Ponto de ônibus  Parada Matemática': {'8022': 39},
   'Ponto de ônibus CRUSP': {'8012': 105, '8022': 105},
   'Ponto de ônibus Reitoria/Bancos': {'8012': 105},
   'Ponto de ônibus Cultura Japonesa': {'8022': 105},
   'Ponto de ônibus Hidráulica': {'8012': 105, '8032': 105},
   'Ponto de ônibus Poli Civil': {'8012': 46, '8022': 46}}},
 'ter': {480: {'Ponto de ônibus Portaria III': {'8012': 168, '8022': 123},
   'Ponto de ônibus FAU II': {'8012': 90, '8022': 77},
   'Ponto de ônibus acesso Vl. Indiana': {'8022': 34},
   'Ponto de ônibus ECA': {'8012': 133},
 

##### Salva Map em Arquivo

In [468]:
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

def encode(s):
    return s.encode('latin1').decode('unicode_escape')

with open(map_file_name, 'w+') as f:
    str_dia_hora_ponto_linha = encode(json.dumps(dia_hora_ponto_linha, indent=4, cls=NpEncoder))
    f.write(str_dia_hora_ponto_linha)
    print(str_dia_hora_ponto_linha)

{
    "seg": {
        "480": {
            "Ponto de ônibus Portaria III": {
                "8012": 168,
                "8022": 123
            },
            "Ponto de ônibus FAU II": {
                "8012": 90,
                "8022": 77
            },
            "Ponto de ônibus acesso Vl. Indiana": {
                "8022": 34
            },
            "Ponto de ônibus ECA": {
                "8012": 133
            },
            "Ponto de ônibus metrô Butantã": {
                "8012": 28,
                "8032": 28
            },
            "Ponto de ônibus Cocesp I": {
                "8012": 128
            },
            "Ponto de ônibus Poli Metalúrgica": {
                "8012": 128
            },
            "Ponto de ônibus Educação": {
                "8022": 28
            },
            "Ponto de ônibus  Parada Matemática": {
                "8022": 39
            },
            "Ponto de ônibus CRUSP": {
                "8012": 105,
                "8022": 1

# Outros

## Add employees and professors
TODO: considerar funcionários e professores estrapolando o horário de entrada da pesquisa

##### Número Pessoas por Dia e Instituto

In [469]:
agrupamento_dia_instituto = {}
for dia in dias:
    agrupamento_dia_instituto[dia] = {}
for aula in aulas_intervalo_horario:
    if aula['unidade'] not in agrupamento_dia_instituto[aula['dia']]:
        agrupamento_dia_instituto[aula['dia']][aula['unidade']] = 0
    agrupamento_dia_instituto[aula['dia']][aula['unidade']] += aula['total']
agrupamento_dia_instituto

{'seg': {'Faculdade de Arquitetura e Urbanismo': 49.61538461538462,
  'Instituto de Física': 83.07692307692308,
  'Faculdade de Economia, Administração, Contabilidade e Atuária': 26.25,
  'Faculdade de Filosofia, Letras e Ciências Humanas': 179.86956521739128,
  'Museu de Arte Contemporânea': 12.0,
  'Faculdade de Medicina Veterinária e Zootecnia': 82.66666666666666,
  'Faculdade de Ciências Farmacêuticas': 67.0,
  'Instituto de Matemática e Estatística': 36.75,
  'Escola de Comunicações e Artes': 113.30555555555556,
  'Instituto de Pesquisas Energéticas e Nucleares': 0.0,
  'Faculdade de Odontologia': 0.0,
  'Escola Politécnica': 256.4705882352941,
  'Instituto de Relações Internacionais': 9.666666666666666,
  'Faculdade de Educação': 27.529411764705884,
  'Instituto de Biociências': 39.05882352941176,
  'Instituto de Química': 74.8,
  'Instituto de Geociências': 65.33333333333334,
  'Instituto de Astronomia, Geofísica e Ciências Atmosféricas': 43.714285714285715,
  'Faculdade de Medi

In [470]:
for dia in agrupamento_dia_instituto:
    for instituto in agrupamento_dia_instituto[dia]:
        if len(porc_circular_por_instituto[porc_circular_por_instituto['instituto'] == instituto]['utiliza_circular_ida']) > 0:
            agrupamento_dia_instituto[dia][instituto] *= porc_circular_por_instituto[porc_circular_por_instituto['instituto'] == instituto]['utiliza_circular_ida'].iloc[0]
        else:
            agrupamento_dia_instituto[dia][instituto] = 0
        agrupamento_dia_instituto[dia][instituto] = int(agrupamento_dia_instituto[dia][instituto])

# Pessoas por instituto que utilizam circular de acordo com a pesquisa
agrupamento_dia_instituto

{'seg': {'Faculdade de Arquitetura e Urbanismo': 49,
  'Instituto de Física': 0,
  'Faculdade de Economia, Administração, Contabilidade e Atuária': 0,
  'Faculdade de Filosofia, Letras e Ciências Humanas': 179,
  'Museu de Arte Contemporânea': 0,
  'Faculdade de Medicina Veterinária e Zootecnia': 0,
  'Faculdade de Ciências Farmacêuticas': 67,
  'Instituto de Matemática e Estatística': 36,
  'Escola de Comunicações e Artes': 113,
  'Instituto de Pesquisas Energéticas e Nucleares': 0,
  'Faculdade de Odontologia': 0,
  'Escola Politécnica': 256,
  'Instituto de Relações Internacionais': 0,
  'Faculdade de Educação': 27,
  'Instituto de Biociências': 39,
  'Instituto de Química': 0,
  'Instituto de Geociências': 65,
  'Instituto de Astronomia, Geofísica e Ciências Atmosféricas': 43,
  'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências': 0,
  'Hospital Universitário': 581,
  'Instituto Oceanográfico': 88,
  'Reitoria Administração Cen